In [ ]:
!pip install transformers
!pip install -q transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.2 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files={'train': 'non_skewed_train.csv', 'test': 'non_skewed_test.csv'})

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: [' id', 'text', 'label'],
        num_rows: 14144
    })
    test: Dataset({
        features: [' id', 'text', 'label'],
        num_rows: 3536
    })
})

In [ ]:
dataset = dataset.remove_columns([' id'])

In [ ]:
dataset['train'][0]

{'text': '<user> ask native americans take', 'label': 1}

In [ ]:
dataset['train'].features.keys()

dict_keys(['text', 'label'])

In [ ]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("facebook/roberta-hate-speech-dynabench-r4-target")

def preprocess_data(examples):
  # take a batch of texts
  text = examples["text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length = 30)
  return encoding

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [ ]:
encoded_dataset = dataset.map(preprocess_data, batched=True)

Map:   0%|          | 0/14144 [00:00<?, ? examples/s]

Map:   0%|          | 0/3536 [00:00<?, ? examples/s]

In [ ]:
train_ds = encoded_dataset["train"].shuffle(seed=42)
val_ds = encoded_dataset["test"].shuffle(seed=42)

In [ ]:
train_ds.set_format("torch")
val_ds.set_format("torch")

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("facebook/roberta-hate-speech-dynabench-r4-target",
                                                           num_labels=2)

config.json:   0%|          | 0.00/816 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [ ]:
batch_size = 8
metric_name = "f1"

In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.9 MB/s eta 0:00:00


In [ ]:
!pip install accelerate -U

In [ ]:
import accelerate

accelerate.__version__

'0.27.2'

In [ ]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"experiment-model-roberta",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # Adjusted batch size
    per_device_eval_batch_size=8,  # Adjusted batch size
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    gradient_accumulation_steps=2,  # Gradient accumulation
    logging_steps=100,  # Adjust as needed
    eval_steps=200,  # Adjust as needed
)

In [ ]:
from transformers import AutoTokenizer, AdamW, get_scheduler
optimizer = AdamW(model.parameters(), lr=args.learning_rate, weight_decay=args.weight_decay)
scheduler = get_scheduler(
    "linear",
    optimizer,
    num_warmup_steps=0,
    num_training_steps=len(train_ds) // args.gradient_accumulation_steps * args.num_train_epochs,
)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
!pip install evaluate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.9 MB/s eta 0:00:00


In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = metric.compute(predictions = predictions, references = labels)

    return acc

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, scheduler),
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.329900,0.347010,0.845305
2,0.306100,0.347226,0.854638
3,0.270600,0.392568,0.858314
4,0.170600,0.540099,0.849548
5,0.145400,0.673259,0.853790


TrainOutput(global_step=4420, training_loss=0.25834383500647223, metrics={'train_runtime': 737.7878, 'train_samples_per_second': 95.854, 'train_steps_per_second': 5.991, 'total_flos': 1090266435648000.0, 'train_loss': 0.25834383500647223, 'epoch': 5.0})

In [ ]:
preds = trainer.predict(val_ds)

In [ ]:
val_ds[0]

{'text': '<user> still making bread',
 'label': tensor(0),
 'input_ids': tensor([    0, 41552, 12105, 15698,   202,   442,  8084,     2,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0])}

In [ ]:
preds

PredictionOutput(predictions=array([[ 2.6112483, -2.237335 ],
       [ 2.4636943, -2.0712168],
       [-3.9363766,  3.4513764],
       ...,
       [ 2.1908553, -1.7712196],
       [-1.4879497,  1.1407368],
       [-2.345418 ,  1.9811877]], dtype=float32), label_ids=array([0, 0, 1, ..., 0, 0, 1]), metrics={'test_loss': 0.39256760478019714, 'test_accuracy': 0.858314479638009, 'test_runtime': 8.458, 'test_samples_per_second': 418.067, 'test_steps_per_second': 52.258})

In [ ]:
pred_vals = np.argmax(preds.predictions, axis=-1)
labels = preds.label_ids
pred_vals, labels

(array([0, 0, 1, ..., 0, 1, 1]), array([0, 0, 1, ..., 0, 0, 1]))

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
print('roc: ', roc_auc_score(labels, pred_vals))
print('f1: ', f1_score(labels, pred_vals))
print('accuracy: ', accuracy_score(labels, pred_vals))

roc:  0.8582771083759053
f1:  0.860484544695071
accuracy:  0.858314479638009


In [ ]:
trainer.save_model("./facebook_roberta")

In [ ]:
!zip -r facebook_roberta.zip facebook_roberta/

  adding: facebook_roberta/ (stored 0%)
  adding: facebook_roberta/training_args.bin (deflated 51%)
  adding: facebook_roberta/model.safetensors (deflated 8%)
  adding: facebook_roberta/tokenizer.json (deflated 72%)
  adding: facebook_roberta/vocab.json (deflated 59%)
  adding: facebook_roberta/special_tokens_map.json (deflated 84%)
  adding: facebook_roberta/config.json (deflated 50%)
  adding: facebook_roberta/merges.txt (deflated 53%)
  adding: facebook_roberta/tokenizer_config.json (deflated 76%)


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' creden

In [ ]:
trainer.push_to_hub('facebook_roberta_olid')

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1709480674.c00f3057d93a.1125.0:   0%|          | 0.00/16.1k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ARC4N3/experiment-model-roberta/commit/78074bc00e9976ce05878a208aa42d947372dcbb', commit_message='facebook_roberta_olid', commit_description='', oid='78074bc00e9976ce05878a208aa42d947372dcbb', pr_url=None, pr_revision=None, pr_num=None)